# Online estimation of curvature and torsion for a contour following task


This notebook aims to provide an example of calculating invariant shape descriptors for the application of contour following. 

The estimated invariants can be used to estimate *the progress* along the trajectory, and serve as *feedforward in the robot controller*. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import invariants_py.reparameterization as reparam
import scipy.interpolate as ip
from invariants_py.class_frenetserret_calculation import OCP_calc_pos
from IPython.display import clear_output


ModuleNotFoundError: No module named 'reparameterization'

### Load example contour data and reparameterize to arclength


In [ ]:
data_location = '../data/contour_coordinates.out'
position_data = np.loadtxt(data_location, dtype='float')
trajectory,time_profile,arclength,nb_samples,stepsize = reparam.reparameterize_positiontrajectory_arclength(position_data)
stepsize_orig = stepsize
arclength_n = arclength/arclength[-1]


Visualization of contour profile

In [ ]:
plt.figure(figsize=(8,3))
plt.axis('equal')
plt.plot(trajectory[:,0],trajectory[:,1],'.-')

### Calculate global curvature and torsion for global trajectory to later compare with

Two steps:
1. first specify optimization problem symbolically
2. calculate invariants given measured trajectory

The optimization problem **can be re-used later** for different measurements (provided they are the same size)

In [ ]:
# specify optimization problem symbolically
FS_calculation_problem = OCP_calc_pos(window_len=nb_samples, 
                                           bool_unsigned_invariants = True, w_pos = 100, 
                                           w_deriv = (10**-12)*np.array([1.0, 1.0, 1.0]), 
                                           w_abs = (10**-5)*np.array([1.0, 1.0]))

# calculate invariants given measurements
invariants, calculate_trajectory, movingframes = FS_calculation_problem.calculate_invariants_global(trajectory,stepsize)


Visualize calculated invariants and corresponding trajectory

In [ ]:
plt.figure(figsize=(8,3))
plt.axis('equal')
plt.plot(trajectory[:,0],trajectory[:,1],'.-')
plt.plot(calculate_trajectory[:,0],calculate_trajectory[:,1],'.-')

f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(8,3))
ax1.plot(arclength_n,invariants[:,0])
ax1.set_title('Velocity [m/-]')
ax1.set_xlabel('s [-]')
ax2.plot(arclength_n,invariants[:,1])
ax2.set_title('Curvature [rad/-]')
ax2.set_xlabel('s [-]')
ax3.plot(arclength_n,invariants[:,2])
ax3.set_title('Torsion [rad/-]')
ax3.set_xlabel('s [-]')



## Simulation of online measurements

The purpose is to test the online calculation separate from a real application.

In [ ]:
# Generate noisy measurements in a specified window
def simulate_noisy_measurements(model_trajectory, current_progress, stepsize, online_window_len):
    
    noise_std = 0.001
    
    progress_values = np.linspace(current_progress, current_progress-online_window_len*stepsize, online_window_len )
    noisy_measurements = np.array([model_trajectory(i) for i in progress_values]) 

    return noisy_measurements + np.random.randn(online_window_len,3)*noise_std


### Test the generation of noisy measurements

In [ ]:
# Parameterize the input trajectory as a spline
knots = np.concatenate(([arclength[0]],[arclength[0]],arclength,[arclength[-1]],[arclength[-1]]))
degree = 3
spline_model_trajectory = ip.BSpline(knots,trajectory,degree)

# Generate noisy measurements
test_measurements = simulate_noisy_measurements(spline_model_trajectory,
                                                current_progress=0.8,stepsize=0.005,online_window_len=20)

# Visualization of noisy measurements on trajectory
plt.figure(figsize=(8,3))
plt.axis('equal')
plt.plot(trajectory[:,0],trajectory[:,1],'.-')
plt.plot(test_measurements[:,0],test_measurements[:,1],'k.-')



## Online estimation of curvature and torsion

Estimate invariants with a receding window approach

1. Specify optimization problem once
2. Re-use the same specification during on-line estimation

In [ ]:
# specify optimization problem symbolically
window_len = 20
stepsize = 0.005
window_increment = 10
FS_online_calculation_problem = OCP_calc_pos(window_len=window_len,
                                                  bool_unsigned_invariants = True, 
                                                  w_pos = 50, w_deriv = (10**-7)*np.array([1.0, 1.0, 1.0]), 
                                                  w_abs = (10**-6)*np.array([1.0, 1.0]))


In [ ]:
# On-line estimation
current_progress = 0.0 + window_len*stepsize
while current_progress <= arclength_n[-1]:

    #print(f"current progress = {current_progress}")
    
    measurements = simulate_noisy_measurements(spline_model_trajectory,current_progress,stepsize,window_len)

    # Calculate invariants in window
    invariants_online, trajectory_online, mf = FS_online_calculation_problem.calculate_invariants_online(measurements,stepsize,window_increment)

    # Visualization
    xvector = np.linspace(current_progress-window_len*stepsize, current_progress , window_len)
    
    clear_output(wait=True)
    
    plt.figure(figsize=(12,8))
    plt.subplot(2,2,1)
    plt.plot(trajectory[:,0],trajectory[:,1],'b.-')
    plt.plot(measurements[:,0],measurements[:,1],'k.')
    plt.plot(trajectory_online[:,0],trajectory_online[:,1],'r')
    
    plt.subplot(2,2,3)
    plt.plot(xvector,(invariants_online[:,0]),'r')
    plt.plot(arclength_n,invariants[:,0],'b')
    plt.plot(0,0)
    plt.title('Velocity [m/-]')
    
    plt.subplot(2,2,2)
    plt.plot(xvector,(invariants_online[:,1]),'r')
    plt.plot(arclength_n,invariants[:,1],'b')
    plt.plot(0,0)
    plt.title('Curvature [rad/-]')
    
    plt.subplot(2,2,4)
    plt.plot(xvector,(invariants_online[:,2]),'r')
    plt.plot(arclength_n,invariants[:,2],'b')
    plt.plot(0,1)
    plt.title('Torsion [rad/-]')

    plt.show(block=False)
    
    
    current_progress = round(current_progress + window_increment*stepsize,3) # start index next window


### Future work 

- now the weights are fixed throughout the estimation, could also be made adaptable
- further investigate alternative regularization terms such as minimum-jerk
- integrate with particle filter to estimate progress along trajectory

